# Notebook for processing of CSV files.

In [101]:
import pandas as pd
import numpy as np

### Load in CSV files into python.

In [102]:
# List your CSV file paths manually
csv_files = ['data\\03_Library SystemCustomers.csv', 'data\\03_Library Systembook.csv']

# Load each CSV into a dataframe and assign to individual variables
df_system_customers = pd.read_csv(csv_files[0])
df_systembook = pd.read_csv(csv_files[1])

### Systembook cleansing

In [3]:
# Drop rows where essential fields are missing
essential_fields = ['Id', 'Books', 'Book checkout', 'Book Returned', 'Customer ID']
df_systembook = df_systembook.dropna(subset=essential_fields).copy()

In [4]:
# Fixing typo fields.
df_systembook['Book checkout'] = df_systembook['Book checkout'].str.replace('32/05/2023', '31/05/2023')
df_systembook['Book checkout'] = df_systembook['Book checkout'].str.replace('10/04/2063', '10/04/2023')

# Strip the extra quotes and convert dates to datetime format
df_systembook['Book checkout'] = df_systembook['Book checkout'].str.replace('"', '')
df_systembook['Book checkout'] = pd.to_datetime(df_systembook['Book checkout'].str.replace('"', ''), errors='coerce', dayfirst=True, format="%d/%m/%Y")
df_systembook['Book Returned'] = pd.to_datetime(df_systembook['Book Returned'], errors='coerce', dayfirst=True, format="%d/%m/%Y")


In [5]:
# Convert Id & Customer ID to integers
df_systembook['Id'] = df_systembook['Id'].astype(int)
df_systembook['Customer ID'] = df_systembook['Customer ID'].astype(int)

In [6]:
# Standardize 'Days allowed to borrow' column, convert "weeks" to "days"
df_systembook['Days allowed to borrow'] = 14 # Quick and dirty, as all are 2 weeks.

In [7]:
# Add some derived fields: Loan duration, if overdue.
df_systembook['Loan duration (days)'] = (df_systembook['Book Returned'] - df_systembook['Book checkout']).dt.days
df_systembook['Overdue'] = np.where((df_systembook['Loan duration (days)'] > df_systembook['Days allowed to borrow']), 'Yes', 'No')

# Flag date errors
df_systembook['Date error'] = df_systembook['Loan duration (days)'] < 0

In [8]:
display(df_systembook)

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID,Loan duration (days),Overdue,Date error
0,1,Catcher in the Rye,2023-02-20,2023-02-25,14,1,5,No,False
1,2,Lord of the rings the two towers,2023-03-24,2023-03-21,14,2,-3,No,True
2,3,Lord of the rings the return of the kind,2023-03-29,2023-03-25,14,3,-4,No,True
3,4,The hobbit,2023-04-02,2023-03-25,14,4,-8,No,True
4,5,Dune,2023-04-02,2023-03-25,14,5,-8,No,True
5,6,Little Women,2023-04-02,2023-05-01,14,1,29,Yes,False
6,7,IT,2023-04-10,2023-04-03,14,6,-7,No,True
7,8,Misery,2023-04-15,2023-04-03,14,7,-12,No,True
8,9,Catch 22,2023-04-15,2023-04-16,14,7,1,No,False
9,10,Animal Farm,2023-04-20,2023-04-24,14,2,4,No,False


### SystemCustomers Cleansing

In [9]:
df_system_customers = df_system_customers.dropna()
df_system_customers['Customer ID'] = df_system_customers['Customer ID'].astype(int)

In [10]:
display(df_system_customers)

,Customer ID,Customer Name
0,1,Jane Doe
1,2,John Smith
2,3,Dan Reeves
4,5,William Holden
5,6,Jaztyn Forest
6,7,Jackie Irving
7,8,Matthew Stirling
8,9,Emory Ted


### Aggregation of the data

In [11]:
# Aggregate the systembook and system customer tables
df_with_names = pd.merge(df_systembook, df_system_customers, on='Customer ID', how='left')
df_with_names['Customer Name'] = df_with_names['Customer Name'].fillna('Unknown Customer')

display(df_with_names)

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID,Loan duration (days),Overdue,Date error,Customer Name
0,1,Catcher in the Rye,2023-02-20,2023-02-25,14,1,5,No,False,Jane Doe
1,2,Lord of the rings the two towers,2023-03-24,2023-03-21,14,2,-3,No,True,John Smith
2,3,Lord of the rings the return of the kind,2023-03-29,2023-03-25,14,3,-4,No,True,Dan Reeves
3,4,The hobbit,2023-04-02,2023-03-25,14,4,-8,No,True,Unknown Customer
4,5,Dune,2023-04-02,2023-03-25,14,5,-8,No,True,William Holden
5,6,Little Women,2023-04-02,2023-05-01,14,1,29,Yes,False,Jane Doe
6,7,IT,2023-04-10,2023-04-03,14,6,-7,No,True,Jaztyn Forest
7,8,Misery,2023-04-15,2023-04-03,14,7,-12,No,True,Jackie Irving
8,9,Catch 22,2023-04-15,2023-04-16,14,7,1,No,False,Jackie Irving
9,10,Animal Farm,2023-04-20,2023-04-24,14,2,4,No,False,John Smith


### Push data to SQL Server

In [13]:
import sqlalchemy
from sqlalchemy import create_engine

# Connection String
conn_str = (
    "mssql+pyodbc://localhost/LibrarySystem"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

engine = create_engine(conn_str)

df_with_names.to_sql('library_records', con=engine, if_exists='replace', index=False)

20